In [1]:
from transformers import pipeline
import pandas as pd

e:\university\TLCN\ChatBox\venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
pipe = pipeline('summarization', model='../bart_laws',
                max_length=200, min_length=50, num_beams=4)

In [4]:
# Đọc dữ liệu từ file CSV hoặc JSON
df = pd.read_csv('../data/csv/processed_medical.csv')  # Thay đổi đường dẫn đến tập dữ liệu của bạn

# Kiểm tra dữ liệu
df.head()




,Title,Detailed Content,Reference Link
0,rào_cản tự_nhiên chống lại nhiễm_trùng về da,da thường chặn các vi_sinh_vật xâm_nhập trừ kh...,https://www.msdmanuals.com/vi-vn/professional/...
1,rào_cản tự_nhiên chống lại nhiễm_trùng về niêm...,nhiều màng_nhầy ngập trong chất tiết có đặc_tí...,https://www.msdmanuals.com/vi-vn/professional/...
2,rào_cản tự_nhiên chống lại nhiễm_trùng về đườn...,đường hô_hấp có các hệ_thống lọc đường thở trê...,https://www.msdmanuals.com/vi-vn/professional/...
3,rào_cản tự_nhiên chống lại nhiễm_trùng về đườn...,các rào_cản trong đường tiêu_hóa bao_gồm ph ac...,https://www.msdmanuals.com/vi-vn/professional/...
4,rào_cản tự_nhiên chống lại nhiễm_trùng về đườn...,các rào_cản về đường sinh_dục tiết_niệu bao_gồ...,https://www.msdmanuals.com/vi-vn/professional/...


In [5]:
df.shape

(12488, 3)

In [ ]:
df['Detailed Content'] = df['Detailed Content'].str.replace('_', ' ', regex=False)
df = df.head(1000)

In [ ]:
# Cấu hình tham số tóm tắt
gen_kwargs = {
    'length_penalty': 0.8,
    'num_beams': 2,
    'max_length': 512,
    'min_length': 200
}

In [8]:
from tqdm import tqdm

# summaries = pipe(list(df['Detailed Content']), batch_size=8)
# df['Summary'] = [summary['summary_text'] for summary in summaries]

# Tạo danh sách chứa các bản tóm tắt
summaries = []

# Số lượng bản tóm tắt cần xử lý
total = len(df['Detailed Content'])

# Tóm tắt dữ liệu theo từng batch và theo dõi tiến độ
for i in tqdm(range(0, total, 16), desc="Processing Summaries"):
    batch = df['Detailed Content'][i:i + 16].tolist()
    try:
        batch_summaries = pipe(batch, **gen_kwargs)
        print(batch_summaries)
        summaries.extend(batch_summaries)
    except Exception as e:
        print(f"Error processing batch starting at index {i}: {e}")

# Thêm các bản tóm tắt vào DataFrame
if len(summaries) != total:
    print(f"Warning: Total number of summaries ({len(summaries)}) does not match number of rows in DataFrame ({total})")

if len(summaries) > 0:
    if 'summary_text' in summaries[0]:
        df['Summary'] = [summary['summary_text'] for summary in summaries]
    else:
        df['Summary'] = [summary['summary'] for summary in summaries]
else:
    print("No summaries generated.")

print(df.head())


Processing Summaries:   0%|          | 1/1561 [04:30<117:10:50, 270.42s/it]


IndexError: index out of range in self

In [ ]:
df